In [ ]:
%matplotlib notebook

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import gsw
from neutral_surfaces import pot_dens_surf

In [ ]:
OCCA_dir = "~/work/data/OCCA/"
xrs = xr.open_dataset(OCCA_dir + "DDsalt.0406annclim.nc")
xrt = xr.open_dataset(OCCA_dir + "DDtheta.0406annclim.nc")

In [ ]:
xrst = xr.merge((xrs, xrt))
xrst = xrst.assign(p=gsw.p_from_z(-xrst.Depth_c, 30))

In [ ]:
xrst

In [ ]:
%%timeit
s, t, p = pot_dens_surf(xrst.salt.values, 
                        xrst.theta.values, 
                        xrst.p.values, 
                        0, 1026, axis=1, tol=1e-4)

eos_option branch:
`148 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)`

main branch, aae2f414, (without the eos option and the use of make_sigma_workers):
timing is virtually identical.


In [ ]:
# Here we discard the unneeded time axis.
s, t, p = pot_dens_surf(xrst.salt.values[0], 
                        xrst.theta.values[0], 
                        xrst.p.values, 
                        0, 1026, axis=0, tol=1e-4)


In [ ]:
print(xrst.salt.values.shape, xrst.theta.values.shape, xrst.p.values.shape)
print(s.shape, t.shape, p.shape)


In [ ]:
fig, ax = plt.subplots()
cs = ax.contourf(xrst.Longitude_t.values, xrst.Latitude_t.values, p)
cbar = fig.colorbar(cs, ax=ax)
cbar.set_label("Pressure [dbar]")
ax.set_title(r"Depth of $\sigma_\theta = 26$ in OCCA")

In [ ]:
%%timeit
s, t, p = pot_dens_surf(xrst.salt.values, 
                        xrst.theta.values, 
                        xrst.p.values, 
                        0, 1026, 
                        eos="gsw",
                        axis=1, tol=1e-4)

eos_option branch:
`116 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)`

In [ ]:
%%timeit
s, t, p = pot_dens_surf(xrst.salt.values, 
                        xrst.theta.values, 
                        xrst.p.values, 
                        (35, 0), -2, 
                        eos="gsw",
                        axis=1, tol=1e-4)

In [ ]:

s, t, p = pot_dens_surf(xrst.salt.values[0], 
                        xrst.theta.values[0], 
                        xrst.p.values, 
                        (35, 0), -2, 
                        eos="gsw",
                        axis=0, tol=1e-4)

The target, -2 kg/m$^3$, is the difference between in-situ density and the
reference density based on S=35, T=0. Call this $x$. Then the specific volume anomaly is 
$$
\delta = \frac{-x}{\rho(S, T, p)\, \rho(35, 0, p)}
$$
Hence, -2 corresponds to roughly 200 centiliters/ton.  This calculation is not quite the same as finding a uniform $\delta$ surface.  Also, in this test, we are ignoring the fact that the model data set is supplying potential temperature, but we should be using in-situ temperature.

In [ ]:
print(xrst.salt.values.shape, xrst.theta.values.shape, xrst.p.values.shape)
print(s.shape, t.shape, p.shape)


In [ ]:
fig, ax = plt.subplots()
cs = ax.contourf(xrst.Longitude_t.values, xrst.Latitude_t.values, p)
cbar = fig.colorbar(cs, ax=ax)
cbar.set_label("Pressure [dbar]")
ax.set_title(r"Depth of $\delta \approx 200$ centiliters/ton in OCCA")